# rpy2
python에서 R 문법 사용하기

해당 노트북은 colab에서 작성됨 (rpy2가 설치되어 있는 상태)

## 간단한 코딩

In [3]:
from rpy2.robjects import r
r('x <- rnorm(100)') # generate x at R
r('y <- x + rnorm(100,sd=0.5)') # generate y at R
r('plot(x,y)') # have R plot them
r('lmout <- lm(y~x)') # run the regression
coef=r('lmout$coefficients')
print(coef)
print(coef[0], coef[1])

(Intercept)           x 
-0.01913136  0.91535206 

-0.019131355988620154 0.9153520576895416


In [4]:
# 파이썬 객체로 받을 수 있음
import numpy as np
np.array(coef)

array([-0.01913136,  0.91535206])

In [ ]:
# 라이브러리 설치
from rpy2.robjects.packages import importr
utils = importr('utils')
utils.install_packages('DirichletReg', repos="https://cloud.r-project.org")

In [6]:
r('library(DirichletReg)')

'Dirichle...,'Formula','tools',...,'datasets','methods','base'


## 셀을 통채로 R 언로 사용하기

In [7]:
%load_ext rpy2.ipython

In [32]:
# 아무 것도 없으면 python cell
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris

iris = load_iris()
iris_python = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_python['target'] = iris.target
iris_python.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [33]:
# R 환경에 python 객체 넣어주기
%R -i iris_python

/usr/local/lib/python3.10/dist-packages/rpy2/robjects/pandas2ri.py:55: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, values in obj.iteritems():


In [34]:
# % 하나면 해당 줄만 R 문법으로 바뀜
# %% 두개면 해당 셀이 R 문법으로 바뀜
%%R

head(iris_python)

  sepal length (cm) sepal width (cm) petal length (cm) petal width (cm) target
0               5.1              3.5               1.4              0.2      0
1               4.9              3.0               1.4              0.2      0
2               4.7              3.2               1.3              0.2      0
3               4.6              3.1               1.5              0.2      0
4               5.0              3.6               1.4              0.2      0
5               5.4              3.9               1.7              0.4      0


In [ ]:
%%R
install.packages("dplyr")
library(dplyr)


In [35]:
%%R
result <- iris_r %>%
  group_by(target) %>%
  summarise(
    sepal_length_max = max(`sepal length (cm)`),
    sepal_length_mean = mean(`sepal length (cm)`),
    petal_length_mean = mean(`petal length (cm)`)
  )

# 결과 출력
print(result)

# A tibble: 3 × 4
  target sepal_length_max sepal_length_mean petal_length_mean
   <int>            <dbl>             <dbl>             <dbl>
1      0              5.8              5.01              1.46
2      1              7                5.94              4.26
3      2              7.9              6.59              5.55


In [36]:
# 파이썬으로 보내기
%R -o result

In [37]:
result

,target,sepal_length_max,sepal_length_mean,petal_length_mean
1,0,5.8,5.006,1.462
2,1,7.0,5.936,4.260
3,2,7.9,6.588,5.552
